<a href="https://colab.research.google.com/github/Mr-Faizan-Asim/Elder-Echo/blob/main/RAG_BASIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain langchain_community langchain-google-genai python-dotenv streamlit langchain_experimental sentence-transformers langchain_chroma langchainhub pypdf rapidocr-onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/Empathy.pdf")
data = loader.load()  # entire PDF is loaded as a single Document
#data

In [6]:
len(data)


47

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  115


In [9]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key="")
vector = embeddings.embed_query("Hello Word")
vector[:5]
vector



[0.04018241912126541,
 -0.03704971820116043,
 -0.04325985908508301,
 -0.010896343737840652,
 -0.00045492153731174767,
 -0.021710863336920738,
 0.01702677272260189,
 -0.0060086906887590885,
 0.029470207169651985,
 -0.00047571954200975597,
 0.06752173602581024,
 0.04668280482292175,
 -0.01687520183622837,
 -0.0642034113407135,
 -0.014988153241574764,
 0.01670013554394245,
 0.013260318897664547,
 0.0020212202798575163,
 -0.02148391306400299,
 -0.016418294981122017,
 0.016205718740820885,
 0.008860348723828793,
 0.00821411982178688,
 -0.021634547039866447,
 0.020011071115732193,
 0.03262202441692352,
 0.0019618261139839888,
 -0.06718710064888,
 -0.03296656161546707,
 0.0338907390832901,
 -0.04732196778059006,
 0.033349912613630295,
 -0.06806593388319016,
 -0.0011030292371287942,
 0.0056602247059345245,
 -0.05940469354391098,
 0.02654408849775791,
 0.008836732245981693,
 -3.7461104511749e-05,
 0.021760495379567146,
 0.0009287774446420372,
 -0.051095981150865555,
 -0.015307459980249405,
 -0.